In [2]:
# ============================================================================
# NOTEBOOK 4: LOAD DATA TO MYSQL
# ============================================================================
# This notebook connects to MySQL and loads all processed data tables safely
# ============================================================================

# %%
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Database configuration
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'user': os.getenv('DB_USER', 'root'),
    'password': os.getenv('DB_PASSWORD'),  # loaded securely
    'database': os.getenv('DB_NAME', 'customer_analytics'),
    'port': int(os.getenv('DB_PORT', 3306))
}

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
sys.path.append('../')

from src.database_connection import DatabaseConnection
from src.database_connection import DatabaseConnection


# %% [markdown]
# # Load Cleaned Data to MySQL Database

# %%
# ==============================
# 1️⃣ DATABASE CONFIGURATION
# ==============================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'icandoit5@A',  # CHANGE IF NEEDED
    'database': 'customer_analytics',
    'port': 3306
}

# Initialize and test connection
db = DatabaseConnection(**DB_CONFIG)
engine = db.create_connection()

if engine is None:
    raise SystemExit("❌ Could not connect to MySQL. Please check DB_CONFIG or MySQL server status.")

# %%
# ==============================
# 2️⃣ CONFIRM DATABASE EXISTS
# ==============================
print("\n⚠️ Make sure you've created the database in MySQL Workbench first:")
print("""
CREATE DATABASE IF NOT EXISTS customer_analytics;
USE customer_analytics;
""")

# %%
# ==============================
# 3️⃣ DEFINE FILE PATHS
# ==============================
file_paths = {
    "customers": "../data/processed/customer_enhanced.csv",
    "rfm_segments": "../data/processed/rfm_segment_summary.csv",
    "clv_tiers": "../data/processed/clv_tier_summary.csv",
    "customer_clusters": "../data/processed/cluster_analysis.csv"
}

# %%
# ==============================
# 4️⃣ LOAD DATA TO MYSQL
# ==============================
for table_name, path in file_paths.items():
    if not os.path.exists(path):
        print(f"⚠️ File not found: {path} — skipping table '{table_name}'")
        continue

    try:
        df = pd.read_csv(path)
        print(f"\n📦 Loading '{table_name}' ...")
        print(f"   Shape: {df.shape}")

        db.load_dataframe_to_db(df, table_name, if_exists='replace')

        # Verify row count
        verify_query = f"SELECT COUNT(*) AS total_rows FROM {table_name}"
        result = db.execute_query(verify_query)

        if not result.empty:
            print(f"✅ Verification: {result['total_rows'][0]:,} rows in '{table_name}' table")
        else:
            print(f"⚠️ Verification failed for '{table_name}'")
    except Exception as e:
        print(f"❌ Error processing '{table_name}': {e}")

# %%
# ==============================
# 5️⃣ VERIFY ALL TABLES IN DATABASE
# ==============================
print("\n" + "=" * 80)
print("VERIFYING TABLES IN DATABASE")
print("=" * 80)

try:
    tables = db.execute_query("SHOW TABLES;")

    if not tables.empty:
        print(tables)
        print("\n📊 Row Counts:")
        for table in tables.iloc[:, 0]:
            count_query = f"SELECT COUNT(*) as count FROM {table}"
            result = db.execute_query(count_query)
            print(f"  {table:<25}: {result['count'][0]:,} rows")
    else:
        print("⚠️ No tables found in the database.")
except Exception as e:
    print(f"❌ Error listing tables: {e}")

# %%
# ==============================
# 6️⃣ SAMPLE TEST QUERIES
# ==============================
print("\n" + "=" * 80)
print("SAMPLE TEST QUERIES")
print("=" * 80)

queries = {
    "Top 10 Customers by Purchase": """
        SELECT customer_id, gender, age, purchase_amount, category, subscription_status
        FROM customers
        ORDER BY purchase_amount DESC
        LIMIT 10;
    """,
    "Revenue by Category": """
        SELECT category,
               COUNT(*) AS purchase_count,
               ROUND(AVG(purchase_amount), 2) AS avg_purchase,
               ROUND(SUM(purchase_amount), 2) AS total_revenue
        FROM customers
        GROUP BY category
        ORDER BY total_revenue DESC;
    """
}

for title, query in queries.items():
    try:
        print(f"\n🔹 {title}")
        result = db.execute_query(query)
        if not result.empty:
            print(result.head(10))
        else:
            print("⚠️ Query returned no results.")
    except Exception as e:
        print(f"❌ Error executing query '{title}': {e}")

# %%
# ==============================
# 7️⃣ COMPLETION
# ==============================
print("\n" + "=" * 80)
print("🎯 DATA LOADING COMPLETE!")
print("=" * 80)
print("""
Next Steps:
1️⃣ Open MySQL Workbench
2️⃣ Run analysis queries from the 'sql/' folder
3️⃣ Verify business insights and dashboards
""")

# Close the DB connection
db.close_connection()



🔗 Connection string preview: mysql+pymysql://root:<hidden>@localhost:3306/customer_analytics
✅ Connected to MySQL database: customer_analytics

⚠️ Make sure you've created the database in MySQL Workbench first:

CREATE DATABASE IF NOT EXISTS customer_analytics;
USE customer_analytics;


📦 Loading 'customers' ...
   Shape: (3900, 27)
✅ Data loaded to table 'customers' successfully!
   Rows loaded: 3,900
✅ Verification: 3,900 rows in 'customers' table

📦 Loading 'rfm_segments' ...
   Shape: (1, 5)
✅ Data loaded to table 'rfm_segments' successfully!
   Rows loaded: 1
✅ Verification: 1 rows in 'rfm_segments' table

📦 Loading 'clv_tiers' ...
   Shape: (4, 6)
✅ Data loaded to table 'clv_tiers' successfully!
   Rows loaded: 4
✅ Verification: 4 rows in 'clv_tiers' table

📦 Loading 'customer_clusters' ...
   Shape: (3, 8)
✅ Data loaded to table 'customer_clusters' successfully!
   Rows loaded: 3
✅ Verification: 3 rows in 'customer_clusters' table

VERIFYING TABLES IN DATABASE
  Tables_in_custo